## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/PV"

In [4]:
waves = np.load(path + "/PV_waves-10sec.npy", allow_pickle= True)


In [5]:
print(waves.shape)


(8657, 40000)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(waves[7],frame_length= 8000, hop_length = 12000)

array([[0.011875, 0.033   , 0.022   , 0.04375 ]])

In [8]:
lbs.feature.zero_crossing_rate(waves[7], frame_length= 8000, hop_length = 12000).shape

(1, 4)

In [9]:
def zero_crossing(array, frame_length= 2048, hl = 512):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 5)) #buradaki ikinci sayı hop lenght e göre değişmeli 

    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:4] = lbs.feature.zero_crossing_rate(array[i],  frame_length = frame_length,
                                                                    hop_length = hl)
        zero_crossing_rate[i][4] = sum(lbs.zero_crossings(array[i]))
    return zero_crossing_rate

        

In [10]:
zcrs = zero_crossing(waves, 8000, hl = 12000) #çok uzun sürüyor, ya datayı kısaltmalıyız ya da hop lenght artmalı

In [11]:
columns_zcr = []
for i in range(zcrs.shape[1]-1):
    columns_zcr.append("zcrs{}".format(i))

    
columns_zcr.append("#ofcrossing")

print(columns_zcr)

['zcrs0', 'zcrs1', 'zcrs2', 'zcrs3', '#ofcrossing']


In [12]:
zcrs[7]

array([1.1875e-02, 3.3000e-02, 2.2000e-02, 4.3750e-02, 1.2660e+03])

In [13]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [14]:
df = dataframe_creator(zcrs, l =5,  columns = columns_zcr)

df.reset_index(drop = True, inplace = True)

In [15]:
df.head()

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing
0,0.022625,0.046375,0.040875,0.035125,1596.0
1,0.016000,0.042125,0.044875,0.049125,1732.0
2,0.017250,0.067750,0.043375,0.051125,2032.0
3,0.021250,0.054625,0.041125,0.039375,1762.0
4,0.016000,0.028875,0.029625,0.046125,1387.0


In [16]:
df.to_csv(path + '/zcrs_PV.csv')

## with Özgün Ozan Nacitarhan code

In [17]:
def zcr(array):
    zcr = np.ones(shape = (array.shape[0],1))
    for i in range(array.shape[0]):
        data_zeros = np.zeros(len(array[i]))
        data_zeros[1:] = array[i][:len(array[i]) - 1]
        zcr[i][0] = (1 / (2 * len(array[i]))) * np.sum(np.abs(np.sign(array[i]) - np.sign(data_zeros)))
        
    return zcr


In [18]:
zcrs_özgün = zcr(waves)

In [19]:
zcrs_özgün.shape

(8657, 1)

In [20]:
zcrs_özgün[0]

array([0.0398875])

In [21]:
np.save(path + "/zcrs_özgün_PV", zcrs_özgün, allow_pickle= True)

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import librosa.display

In [24]:
lbs.feature.chroma_stft(waves[0], sr = 4000, hop_length = 12000).shape

(12, 4)

In [25]:
# we are taking time average of each pitch values.
def chromagram_average(array, sampling_rate = 256):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= sampling_rate, hop_length = 12000)[j].mean()
    return pitch

In [26]:
chro = chromagram_average(waves, sampling_rate = 4000) # bu da çok uzun sürüyor.

In [27]:
print(chro.shape)

(8657, 12)


In [28]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [29]:
col_chro = []
for i in range(chro.shape[1]):
    col_chro.append(lst[i])
    
print(col_chro)
print(len(col_chro))

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
12


In [30]:
ch_df = dataframe_creator(chro,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.629012,0.525487,0.600465,0.710609,0.689672,0.623033,0.60419,0.616193,0.755655,0.824863,0.689838,0.638675
1,0.601472,0.753536,0.820504,0.481287,0.281164,0.205453,0.206726,0.305635,0.368947,0.278555,0.373549,0.555992
2,0.34394,0.293521,0.452866,0.549072,0.554083,0.207506,0.185472,0.367123,0.32174,0.398516,0.474997,0.487707
3,0.698843,0.570073,0.574082,0.473231,0.341271,0.428014,0.532095,0.427049,0.366476,0.479816,0.68783,0.800538
4,0.361276,0.18809,0.0868,0.138867,0.267305,0.458633,0.546011,0.807571,0.627838,0.361924,0.176646,0.248592
...,...,...,...,...,...,...,...,...,...,...,...,...
8652,0.675475,0.746057,0.735981,0.7051,0.705708,0.710661,0.726375,0.719125,0.772572,0.705887,0.675076,0.649648
8653,0.539034,0.624321,0.928424,0.429256,0.350291,0.32745,0.350677,0.382048,0.382901,0.460927,0.500787,0.622448
8654,0.760542,0.769394,0.770742,0.820602,0.754071,0.802725,0.89479,0.833151,0.793919,0.800624,0.781006,0.716278
8655,0.621852,0.606681,0.723906,0.719411,0.704855,0.721545,0.7063,0.734808,0.800435,0.601481,0.583356,0.548433


In [31]:
ch_df.to_csv(path + "/chromogram_PV.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [32]:
def tempo_average(array): # 192,79
    means= []
    for i in range(array.shape[0]): #192
        means.append(array[i].mean())
    return np.array(means) # 192
        

In [33]:
def tempogram(array = 0, hop_lenght = 12000, sampling_rate = 256):
    
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sampling_rate, 
                                        lag =3)

    times = librosa.times_like(oenv, sr=sampling_rate, hop_length=12000)

    tempogram = librosa.feature.tempogram(onset_envelope = oenv, sr = sampling_rate,
                                              hop_length = hop_lenght, win_length = 192 )
    # Estimate the global tempo for display purposes
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sampling_rate,
                                   #hop_length=hop_length)[0]
    return tempogram

In [34]:
tempogram(waves[0]).shape

(192, 79)

In [35]:
tempogram(waves[0])[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [36]:
tempogram(waves[0])[1]

array([0.81880754, 0.81876209, 0.81873756, 0.81873346, 0.81874927,
       0.81878451, 0.81883866, 0.81891123, 0.81900173, 0.81910968,
       0.81923461, 0.81937606, 0.81953359, 0.81970674, 0.81989511,
       0.82009827, 0.82031584, 0.82054744, 0.82079268, 0.82105122,
       0.82132272, 0.82160685, 0.8219033 , 0.82221178, 0.822532  ,
       0.82286369, 0.8232066 , 0.82356048, 0.82392511, 0.82430026,
       0.82468574, 0.82508135, 0.82548692, 0.82590227, 0.82632725,
       0.82676172, 0.82720554, 0.82765858, 0.82812074, 0.8285919 ,
       0.82907198, 0.82956089, 0.83005854, 0.83056489, 0.83107986,
       0.83160341, 0.83213549, 0.83267607, 0.83322513, 0.83378263,
       0.83434857, 0.83492294, 0.83550574, 0.83609698, 0.83669667,
       0.83730482, 0.83792146, 0.83854662, 0.83918033, 0.83982263,
       0.84047357, 0.84113318, 0.84180153, 0.84247867, 0.84316466,
       0.84385956, 0.84456344, 0.84527638, 0.84599844, 0.8467297 ,
       0.84747024, 0.84822015, 0.84897951, 0.84974841, 0.85052

In [37]:
tempo_average(tempogram(waves[0]))

array([ 1.00000000e+00,  8.31099507e-01,  5.58455997e-01,  4.00874589e-01,
        3.69644662e-01,  3.86257191e-01,  4.15605860e-01,  4.18401802e-01,
        3.89638678e-01,  3.71667471e-01,  4.38155728e-01,  5.76515964e-01,
        6.64286380e-01,  6.30245251e-01,  5.07269188e-01,  3.96937332e-01,
        3.45189943e-01,  3.37674179e-01,  3.41692714e-01,  3.38774684e-01,
        3.12796760e-01,  2.91901975e-01,  3.24721365e-01,  4.46167200e-01,
        5.67128820e-01,  5.26257821e-01,  3.65444806e-01,  2.36930013e-01,
        2.05308513e-01,  2.11457002e-01,  2.34933932e-01,  2.53515977e-01,
        2.57130849e-01,  2.40359717e-01,  2.57157423e-01,  3.27726568e-01,
        3.95945797e-01,  3.89173447e-01,  3.23077849e-01,  2.52899896e-01,
        2.06095177e-01,  1.81056465e-01,  1.83345971e-01,  1.97942394e-01,
        1.93437645e-01,  1.74808907e-01,  1.59672497e-01,  1.71025967e-01,
        2.02515652e-01,  2.15896788e-01,  1.88894009e-01,  1.55174787e-01,
        1.34418538e-01,  

# tempogram(waves[0])[0]

In [38]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 192))
    for i in range(array.shape[0]): #8bin civarı
        
        _x = tempogram(array[i]) #1,4000   #192,79
        #print(_x.shape)
        final[i] = tempo_average(_x) #192
        
        
    return final

In [39]:
temp = temp_values(waves) #çok uzun sürüyor 


In [40]:
temp.shape

(8657, 192)

In [41]:
columns_t = []
for i in range(temp.shape[1]):
    columns_t.append("tempogram_{}".format(i))
print(columns_t)

['tempogram_0', 'tempogram_1', 'tempogram_2', 'tempogram_3', 'tempogram_4', 'tempogram_5', 'tempogram_6', 'tempogram_7', 'tempogram_8', 'tempogram_9', 'tempogram_10', 'tempogram_11', 'tempogram_12', 'tempogram_13', 'tempogram_14', 'tempogram_15', 'tempogram_16', 'tempogram_17', 'tempogram_18', 'tempogram_19', 'tempogram_20', 'tempogram_21', 'tempogram_22', 'tempogram_23', 'tempogram_24', 'tempogram_25', 'tempogram_26', 'tempogram_27', 'tempogram_28', 'tempogram_29', 'tempogram_30', 'tempogram_31', 'tempogram_32', 'tempogram_33', 'tempogram_34', 'tempogram_35', 'tempogram_36', 'tempogram_37', 'tempogram_38', 'tempogram_39', 'tempogram_40', 'tempogram_41', 'tempogram_42', 'tempogram_43', 'tempogram_44', 'tempogram_45', 'tempogram_46', 'tempogram_47', 'tempogram_48', 'tempogram_49', 'tempogram_50', 'tempogram_51', 'tempogram_52', 'tempogram_53', 'tempogram_54', 'tempogram_55', 'tempogram_56', 'tempogram_57', 'tempogram_58', 'tempogram_59', 'tempogram_60', 'tempogram_61', 'tempogram_62', '

In [42]:
len(columns_t)

192

In [43]:
df_t = dataframe_creator(temp,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 192)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_0,tempogram_1,tempogram_2,tempogram_3,tempogram_4,tempogram_5,tempogram_6,tempogram_7,tempogram_8,tempogram_9,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,1.0,0.8311,0.558456,0.400875,0.369645,0.386257,0.415606,0.418402,0.389639,0.371667,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,1.0,0.768033,0.441054,0.331387,0.387299,0.528746,0.645348,0.613041,0.467121,0.374111,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
2,1.0,0.835704,0.554074,0.383544,0.337688,0.334651,0.35937,0.403639,0.463697,0.537177,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.935448,0.837743,0.778187,0.761626,0.758663,0.749762,0.722433,0.688283,0.661175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.869964,0.707931,0.651308,0.685073,0.756092,0.765803,0.722458,0.685163,0.680484,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8652,1.0,0.732823,0.34829,0.170994,0.193337,0.239143,0.244301,0.28294,0.418028,0.580885,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8653,1.0,0.758206,0.402393,0.209776,0.191685,0.202634,0.187853,0.203558,0.266254,0.356037,...,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0
8654,1.0,0.781983,0.428584,0.204175,0.150491,0.160826,0.196889,0.21917,0.294314,0.373395,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8655,1.0,0.770552,0.379235,0.144263,0.102502,0.152174,0.218006,0.250193,0.304236,0.396733,...,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df_t['tempogram_188'][10]

1.1777135612269726e-17

In [45]:
df_t.to_csv(path + '/tempogram_PV.csv')

In [46]:
df_t.isin([0]).sum(axis=0)

tempogram_0      0
tempogram_1      0
tempogram_2      0
tempogram_3      0
tempogram_4      0
                ..
tempogram_187    0
tempogram_188    0
tempogram_189    0
tempogram_190    0
tempogram_191    0
Length: 192, dtype: int64

In [47]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.022625,0.046375,0.040875,0.035125,1596.0,0.629012,0.525487,0.600465,0.710609,0.689672,0.623033,0.60419,0.616193,0.755655,0.824863,0.689838,0.638675
1,0.016000,0.042125,0.044875,0.049125,1732.0,0.601472,0.753536,0.820504,0.481287,0.281164,0.205453,0.206726,0.305635,0.368947,0.278555,0.373549,0.555992
2,0.017250,0.067750,0.043375,0.051125,2032.0,0.34394,0.293521,0.452866,0.549072,0.554083,0.207506,0.185472,0.367123,0.32174,0.398516,0.474997,0.487707
3,0.021250,0.054625,0.041125,0.039375,1762.0,0.698843,0.570073,0.574082,0.473231,0.341271,0.428014,0.532095,0.427049,0.366476,0.479816,0.68783,0.800538
4,0.016000,0.028875,0.029625,0.046125,1387.0,0.361276,0.18809,0.0868,0.138867,0.267305,0.458633,0.546011,0.807571,0.627838,0.361924,0.176646,0.248592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8652,0.012875,0.022625,0.023500,0.028250,872.0,0.675475,0.746057,0.735981,0.7051,0.705708,0.710661,0.726375,0.719125,0.772572,0.705887,0.675076,0.649648
8653,0.004375,0.014000,0.013750,0.015125,744.0,0.539034,0.624321,0.928424,0.429256,0.350291,0.32745,0.350677,0.382048,0.382901,0.460927,0.500787,0.622448
8654,0.015125,0.025000,0.024000,0.019250,847.0,0.760542,0.769394,0.770742,0.820602,0.754071,0.802725,0.89479,0.833151,0.793919,0.800624,0.781006,0.716278
8655,0.005250,0.022625,0.021000,0.024125,746.0,0.621852,0.606681,0.723906,0.719411,0.704855,0.721545,0.7063,0.734808,0.800435,0.601481,0.583356,0.548433


In [48]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,0.022625,0.046375,0.040875,0.035125,1596.0,0.629012,0.525487,0.600465,0.710609,0.689672,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,0.016000,0.042125,0.044875,0.049125,1732.0,0.601472,0.753536,0.820504,0.481287,0.281164,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
2,0.017250,0.067750,0.043375,0.051125,2032.0,0.34394,0.293521,0.452866,0.549072,0.554083,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.021250,0.054625,0.041125,0.039375,1762.0,0.698843,0.570073,0.574082,0.473231,0.341271,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.016000,0.028875,0.029625,0.046125,1387.0,0.361276,0.18809,0.0868,0.138867,0.267305,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8652,0.012875,0.022625,0.023500,0.028250,872.0,0.675475,0.746057,0.735981,0.7051,0.705708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8653,0.004375,0.014000,0.013750,0.015125,744.0,0.539034,0.624321,0.928424,0.429256,0.350291,...,-0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0
8654,0.015125,0.025000,0.024000,0.019250,847.0,0.760542,0.769394,0.770742,0.820602,0.754071,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8655,0.005250,0.022625,0.021000,0.024125,746.0,0.621852,0.606681,0.723906,0.719411,0.704855,...,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
final_df.to_csv(path + '/final_sp_PV.csv')